In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import os
from datetime import datetime, UTC
import analytiq_data as ad
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
import docrouter_app.payments

In [4]:
ad.common.setup()
await docrouter_app.payments.init_payments_env()

In [5]:
#await docrouter_app.payments.delete_all_payments_customers(dryrun=False)

In [6]:
org_id = "6795345439604beca2b2808d"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
stripe_customer = await docrouter_app.payments.get_payments_customer(org_id=org_id)
stripe_customer

In [ ]:
stripe_customer.id

In [ ]:
subscription = await docrouter_app.payments.get_subscription(stripe_customer.id)
subscription

In [32]:
await docrouter_app.payments.get_stripe_usage(org_id=org_id)

2025-07-03 14:57:00,935 - docrouter_app.payments - INFO - Current period start: 2025-07-01T10:51:10
2025-07-03 14:57:00,936 - docrouter_app.payments - INFO - Current period end: 2025-08-01T10:51:10
2025-07-03 14:57:00,940 - docrouter_app.payments - INFO - Using stored prorated included usage for 6795345439604beca2b2808d: 500


{'total_usage': 540,
 'included_usage': 500,
 'overage_usage': 40,
 'remaining_included': 0,
 'period_start': datetime.datetime(2025, 7, 1, 10, 51, 10),
 'period_end': datetime.datetime(2025, 8, 1, 10, 51, 10),
 'subscription_type': 'team',
 'usage_unit': 'spu'}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [31]:
await docrouter_app.payments.handle_usage_record(org_id=org_id, spus=400, operation="test")

2025-07-03 14:56:53,115 - docrouter_app.payments - INFO - handle_usage_record() called with org_id: 6795345439604beca2b2808d, spus: 400, operation: test, source: backend
2025-07-03 14:56:53,307 - docrouter_app.payments - INFO - Stripe customer found for org_id: 6795345439604beca2b2808d: cus_SbGxOTbDNTq6wa


{'org_id': '6795345439604beca2b2808d',
 'stripe_customer_id': 'cus_SbGxOTbDNTq6wa',
 'spus': 400,
 'operation': 'test',
 'source': 'backend',
 'timestamp': datetime.datetime(2025, 7, 3, 18, 56, 53, 508442),
 'billing_period_id': None,
 '_id': ObjectId('6866d2752acc40c71544cf8c')}

In [ ]:
await docrouter_app.payments.set_subscription_type(stripe_customer.id, "individual")

In [38]:
org_id = "6859696f6ebcee0b712b9ac6"

In [ ]:
await docrouter_app.payments.sync_payments_customer(org_id=org_id)

In [ ]:
await docrouter_app.payments.delete_payments_customer(org_id=org_id, force=False)

In [ ]:
await docrouter_app.payments.get_tier_config()